# Integración de Datos Datawarehouse

## Importación de librerías

In [2]:
import pandas as pd
import numpy as np
import os
import re

# Ambiente de Google Colab de ser necesario

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# Ruta de Archivos

In [4]:
base_path = os.getcwd()
path = os.path.join(base_path, "CLEANED_DATA")

In [5]:
os.listdir(path)

['.ipynb_checkpoints',
 'ci.csv',
 'ci_dl_provincia_canton.csv',
 'ci_lca.csv',
 'cnc.csv',
 'correos_corregidos.csv',
 'doc_name_pcci.csv',
 'doc_name_pcoc.csv',
 'main_phone_pcci.csv',
 'oc.csv',
 'oc.xlsx',
 'oc_cc.csv',
 'oc_cl.csv',
 'oc_dl_area_especialidad.csv',
 'oc_dl_familia_sector_perfil.csv',
 'oc_dl_provincia_canton.csv',
 'oec.csv',
 'oec_dl_familia_sector_perfil.csv',
 'oec_dl_provincia_canton.csv',
 'oec_dr.csv',
 'pcci.csv',
 'pcoc.csv',
 'pcoec.csv',
 'provincia_canton_parroquia.xlsx',
 'Ubicacion.csv']

### Dimension Fecha OC

Se debe desarrollar en base a el año el cual se haya obtenido el certificado del curso o perfil OC.

En caso de no existir un año registrado en el certificado en la tabla **PCOC** se deberá tomar como referencia el año de resolución de la Razon social de la tabla **OC**

In [67]:
# Cargar los archivos necesarios de la ubicación de CLEANED DATA
df_pcoc = pd.read_csv(os.path.join(path,'pcoc.csv'))

In [68]:
# Se extraera el año de la columna Numero_certificado.
# Para esto se buscará porciones de texto que contengan los años del 2000 - 2022 (año actual)
year_list = [str(x) for x in range(2022,2009,-1)]

In [69]:
def get_year(x):
    for year in year_list:
        if re.search('-'+year+'\s|'+year+'-|-'+year+'$',x):
            return year
    return 'sin_anio'

In [70]:
df_pcoc['anio'] = df_pcoc['Numero_Certificado'].apply(get_year)


In [71]:
df_oc = pd.read_csv(os.path.join(path,'oc.csv'))


In [72]:
df_extract_0_oc = df_oc[['Razon_Social','Fecha_Resolucion']]

In [73]:
df_pcoc_1 = pd.merge(df_pcoc,df_extract_0_oc,left_on=['Razon_Social_OC'],right_on=['Razon_Social'],how="left") 


In [74]:
df_pcoc_1['anio'].value_counts()

sin_anio    201063
2019         32718
2018         21010
2017           209
2022           134
2020            49
2021            29
2016             2
2010             1
Name: anio, dtype: int64

In [75]:
def get_year_by_oc(x,y):
    if x=='sin_anio':
        if y != 'Sin Fecha Resolución' and not pd.isna(y):
            return y[-4:]
    else:
        return x

In [76]:
df_pcoc_1['anio'] = df_pcoc_1.apply(lambda x: get_year_by_oc(x['anio'], x['Fecha_Resolucion']), axis=1)

In [220]:
df_pcoc_1[
    (df_pcoc_1['anio']=='sin_anio') & 
    (df_pcoc_1['Fecha_Resolucion']!='Sin Fecha Resolución') &
    (df_pcoc_1['Fecha_Resolucion'].notnull())
    ]['anio'] = df_pcoc_1[
    (df_pcoc_1['anio']=='sin_anio') & 
    (df_pcoc_1['Fecha_Resolucion']!='Sin Fecha Resolución') &
    (df_pcoc_1['Fecha_Resolucion'].notnull())
    ]['Fecha_Resolucion'].str.slice(start=-4)

C:\Users\sjime\AppData\Local\Temp\ipykernel_14588\3272475287.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_pcoc_1[


In [77]:
df_pcoc_1['anio'].value_counts()

2022    78061
2021    77987
2019    33250
2018    22116
2020     7827
2017      209
2016        2
2010        1
Name: anio, dtype: int64

# Datawarehouse

In [29]:
#path_datawarehouse = '/content/drive/MyDrive/Trabajo_de_Integracion_Curricular/CODE/DataWareHouse/'
path_datawarehouse = os.path.join(base_path, "DATAWAREHOUSE")
path_datamart_oc = 'datamart_oc'
path_datamart_oec = 'datamart_oec'
path_datamart_ci = 'datamart_ci'

 ## Tabla Ubicacion

In [38]:
df_ubicacion = pd.read_csv(os.path.join(path_datawarehouse, 'ubicacion.csv'))

# Datamart OC

## Dimensión Razon Social OC

In [40]:
df_oc = pd.read_csv(os.path.join(path,'oc.csv'))
df_oc_ug = pd.read_csv(os.path.join(path,'oc_dl_provincia_canton.csv'))
df_oc_ug['Estado']='DESCONOCIDO'
df_oc_ug['Razon_Social']=df_oc_ug['Nombre']
df_razon_social_oc = pd.concat([df_oc[['Razon_Social','Estado','Canton']],df_oc_ug[['Razon_Social','Estado','Canton']]],axis=0)
df_razon_social_oc = df_razon_social_oc.drop_duplicates(subset=['Razon_Social','Canton'],keep='first')
df_razon_social_oc=df_razon_social_oc.rename(columns={'Razon_Social':'nombre','Canton':'canton','Estado':'estado'})
df_razon_social_oc = pd.merge(df_razon_social_oc,df_ubicacion,on='canton',how="left")
df_razon_social_oc.pop('id_ubicacion')
df_razon_social_oc = df_razon_social_oc.fillna({'provincia':'Sin Provincia','Provincia':'Sin Provincia'})
df_razon_social_oc = df_razon_social_oc.rename_axis('id_oc').reset_index().astype('object')
df_razon_social_oc.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 746 entries, 0 to 745
Data columns (total 5 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   id_oc      746 non-null    object
 1   nombre     746 non-null    object
 2   estado     746 non-null    object
 3   canton     746 non-null    object
 4   provincia  746 non-null    object
dtypes: object(5)
memory usage: 29.3+ KB


In [ ]:
df_razon_social_oc.to_csv(os.path.join(path_datawarehouse,path_datamart_oc,'dim_razon_social_oc.csv'),index=False)

## Dimensión Curso_Perfil

In [54]:
df_oc_cl = pd.read_csv(os.path.join(path,'oc_dl_familia_sector_perfil.csv'))
df_oc_cl_2 = pd.read_csv(os.path.join(path,'oc_cl.csv'))
df_oc_cl_2.pop('RUC_o_Codigo')
df_oc_cl_2 = df_oc_cl_2.rename(columns={'Razon_Social':'Nombre'})
df_oc_cl = pd.concat([df_oc_cl,df_oc_cl_2],axis=0)
df_oc_cl = df_oc_cl.drop_duplicates()
df_oc_cl['modalidad'] = np.NaN
df_oc_cl['carga_horaria'] = np.NaN
df_oc_cl['tipo'] = 'perfil'
df_oc_cc = pd.read_csv(os.path.join(path,'oc_cc.csv'))
df_oc_cc.pop('documento')
df_oc_cc['tipo'] = 'curso'

In [61]:


data_curso_perfil = np.concatenate((df_oc_cc.values,df_oc_cl.values), axis=0)
df_curso_perfil = pd.DataFrame(data=data_curso_perfil,columns=['razon_social','area_familia','especialidad_sector','curso_perfil','modalidad','carga_horaria','tipo'])
df_curso_perfil.insert(0,'tipo',df_curso_perfil.pop('tipo'))
df_curso_perfil = df_curso_perfil.rename_axis('id_curso_perfil').reset_index().astype('object')
df_curso_perfil.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5173 entries, 0 to 5172
Data columns (total 8 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   id_curso_perfil      5173 non-null   object
 1   tipo                 5173 non-null   object
 2   razon_social         5173 non-null   object
 3   area_familia         5173 non-null   object
 4   especialidad_sector  5173 non-null   object
 5   curso_perfil         5173 non-null   object
 6   modalidad            4873 non-null   object
 7   carga_horaria        4873 non-null   object
dtypes: object(8)
memory usage: 323.4+ KB


In [62]:
df_curso_perfil.to_csv(os.path.join(path_datawarehouse,path_datamart_oc,'dim_curso_perfil.csv'),index=False)

## Tabla de Hechos

In [66]:
df_pcoc = pd.read_csv(os.path.join(path,'pcoc.csv'))
df_fact_oc = df_pcoc.groupby(['Razon_Social_OC','Nombre_Curso_Perfil']).count().reset_index().iloc[:,:3]
df_fact_oc = df_fact_oc.rename(columns={'Numero_Documento':'num_cap_cer'})
df_fact_oc = pd.merge(df_fact_oc,df_razon_social_oc,left_on=['Razon_Social_OC'],right_on=['nombre'],how="left")
df_fact_oc = pd.merge(df_fact_oc,df_curso_perfil,left_on=['Razon_Social_OC','Nombre_Curso_Perfil'],right_on=['razon_social','curso_perfil'],how="left")
#df_fact_oc = df_fact_oc[['Razon_Social_OC','Razon_Social','nombre_oc','Estado','Nombre_Curso_Perfil','curso_perfil','num_cap_cer']]
df_fact_oc = df_fact_oc[['id_oc','id_curso_perfil','num_cap_cer']]
df_fact_oc.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4820 entries, 0 to 4819
Data columns (total 3 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   id_oc            4461 non-null   object
 1   id_curso_perfil  1381 non-null   object
 2   num_cap_cer      4820 non-null   int64 
dtypes: int64(1), object(2)
memory usage: 150.6+ KB


In [ ]:
df_demanda_oc.to_csv(path_datawarehouse+path_datamart_oc+'/Demanda_OC.csv',index=False)

# Dataframe OEC

In [ ]:
os.mkdir(path_datawarehouse+path_datamart_oec,0o666)

## Tabla Razon Social OEC

In [ ]:
df_oec_extra = pd.read_csv(path+'OEC_Provincia_Canton.csv')
df_oec_extra

,Nombre,Provincia,Cantón
0,GWG S.A.S. B.I.C.,AZUAY,CUENCA
1,GREEN WORK CONSULTORES,AZUAY,CUENCA
2,ASOCIACION DE CORREDORES DE BIENES RAICES ACBI...,AZUAY,CUENCA
3,EMPRESA ELECTRICA REGIONAL CENTRO SUR CA,AZUAY,CUENCA
4,CAMARA DE LA PEQUEÑA INDUSTRIA DEL AZUAY,AZUAY,CUENCA
...,...,...,...
237,CORDOVA SUAREZ MANOLO ALEXANDER,TUNGURAHUA,AMBATO
238,ABKREA INGENIERIA CIA. LTDA.,TUNGURAHUA,AMBATO
239,L'ESCOFFIER,TUNGURAHUA,AMBATO
240,CECFECONSULTORES C.L.,TUNGURAHUA,AMBATO


In [ ]:
df_oec_2 = pd.DataFrame()
df_oec_2['razon_social_oec'] = df_oec_extra['Nombre']
df_oec_2['estado']='Desconocido'
df_oec_2['provincia']=df_oec_extra['Provincia']
df_oec_2['canton']=df_oec_extra['Cantón']
df_oec_2

,razon_social_oec,estado,provincia,canton
0,GWG S.A.S. B.I.C.,Desconocido,AZUAY,CUENCA
1,GREEN WORK CONSULTORES,Desconocido,AZUAY,CUENCA
2,ASOCIACION DE CORREDORES DE BIENES RAICES ACBI...,Desconocido,AZUAY,CUENCA
3,EMPRESA ELECTRICA REGIONAL CENTRO SUR CA,Desconocido,AZUAY,CUENCA
4,CAMARA DE LA PEQUEÑA INDUSTRIA DEL AZUAY,Desconocido,AZUAY,CUENCA
...,...,...,...,...
237,CORDOVA SUAREZ MANOLO ALEXANDER,Desconocido,TUNGURAHUA,AMBATO
238,ABKREA INGENIERIA CIA. LTDA.,Desconocido,TUNGURAHUA,AMBATO
239,L'ESCOFFIER,Desconocido,TUNGURAHUA,AMBATO
240,CECFECONSULTORES C.L.,Desconocido,TUNGURAHUA,AMBATO


In [ ]:
df_oec = pd.read_csv(path+'oec.csv')
df_razon_social_oec = df_oec[['Razon_Social','Estado','Canton']]
df_razon_social_oec=df_razon_social_oec.rename(columns={'Razon_Social':'razon_social_oec','Canton':'canton','Estado':'estado'})

df_razon_social_oec.to_csv(path_datawarehouse+path_datamart_oec+'/'+'Razon_Social_OEC.csv',index=False)
df_razon_social_oec = pd.merge(df_razon_social_oec,df_ubicacion,on='canton',how="left")
df_razon_social_oec.pop('id_ubicacion')
df_razon_social_oec = df_razon_social_oec.fillna({'provincia':'Sin Provincia'})
df_razon_social_oec = pd.concat([df_razon_social_oec,df_oec_2])
df_razon_social_oec = df_razon_social_oec.drop_duplicates(subset=['razon_social_oec'], keep='last')
df_razon_social_oec = df_razon_social_oec.rename_axis('id_oec').reset_index().astype('object')
df_razon_social_oec.to_csv(path_datawarehouse+path_datamart_oec+'/razon_social_oec.csv',index=False)

## Tabla Perfil

In [ ]:
oec_dr = pd.read_csv(path+'oec_dr.csv')
oec_dr

,RUC_o_Codigo,Razon_Social,Perfil,Esquema_de_Certificacion
0,1801742642001,NAVAS ESCOBAR FAUSTO ENRIQUE,ASESORÍA DE IMAGEN,ASESORÍA DE IMAGEN
1,1801742642001,NAVAS ESCOBAR FAUSTO ENRIQUE,MAQUILLAJE,MAQUILLAJE
2,0992864753001,SESOCORPSA S.A.,Sin Perfil,Sin Esquema de Certificación
3,1793005489001,ESCUELA PROFESIONAL Y TECNICA CODADEPRO C.A.,ADMINISTRACIÓN DE EMPRESAS,ADMINISTRACIÓN DE EMPRESAS
4,1793005489001,ESCUELA PROFESIONAL Y TECNICA CODADEPRO C.A.,ARMADO DE ESTRUCTURAS METÁLICAS,ARMADO DE ESTRUCTURAS METÁLICAS
...,...,...,...,...
992,0591755080001,COOPERACIÓN Y RESPALDO PARA LA EDUCACIÓN COORE...,VENTA DE PRODUCTOS,COMERCIO MINORISTA
993,0591755080001,COOPERACIÓN Y RESPALDO PARA LA EDUCACIÓN COORE...,VENTA DE PRODUCTOS,VENTA EN MOSTRADOR
994,0591755080001,COOPERACIÓN Y RESPALDO PARA LA EDUCACIÓN COORE...,VENTA DE PRODUCTOS,VENTA Y DESPACHO DE PRODUCTOS
995,0160001240001,UNIVERSIDAD DE CUENCA,Sin Perfil,Sin Esquema de Certificación


In [ ]:
df_perfil = oec_dr[['Razon_Social','Perfil','Esquema_de_Certificacion']]
df_perfil = df_perfil.rename_axis('id_perfil').reset_index().astype('object')
len(oec_dr['Perfil'].unique())

128

### Downloaded Data

In [ ]:
df_perfil = pd.read_csv(path+'OEC_Familia_Sector_Perfil.csv')
df_perfil = df_perfil.rename_axis('id_perfil').reset_index().astype('object')
df_perfil = df_perfil.rename(columns={'Perfil':'Perfil_Perfil','Nombre':'Perfil_Razon_Social_OEC'})
len(oec_dr['Perfil'].unique())
df_perfil.to_csv(path_datawarehouse+path_datamart_oec+'/perfil.csv',index=False)

## Tabla Fecha

## Tabla Hechos OEC

In [ ]:
df_pcoec = pd.read_csv(path+'pcoec.csv')
df_demanda_oec = df_pcoec.groupby(['OEC','Perfil','Fecha_Certificacion']).count().reset_index().iloc[:,:4]
df_demanda_oec = df_demanda_oec.rename(columns={'Nombres':'num_cer'})
df_demanda_oec = pd.merge(df_demanda_oec,df_razon_social_oec,left_on=['OEC'],right_on=['razon_social_oec'],how="left")
df_demanda_oec

,OEC,Perfil,Fecha_Certificacion,num_cer,id_oec,razon_social_oec,estado,canton,provincia
0,ABALCO HARO CRISTIAN PATRICIO,ASISTENCIA DE CONTABILIDAD,01-06-2022,6,79,ABALCO HARO CRISTIAN PATRICIO,Desconocido,GUAYAQUIL,GUAYAS
1,ABALCO HARO CRISTIAN PATRICIO,ASISTENCIA DE CONTABILIDAD,02-08-2022,1,79,ABALCO HARO CRISTIAN PATRICIO,Desconocido,GUAYAQUIL,GUAYAS
2,ABALCO HARO CRISTIAN PATRICIO,ASISTENCIA DE CONTABILIDAD,04-04-2022,3,79,ABALCO HARO CRISTIAN PATRICIO,Desconocido,GUAYAQUIL,GUAYAS
3,ABALCO HARO CRISTIAN PATRICIO,ASISTENCIA DE CONTABILIDAD,05-03-2022,4,79,ABALCO HARO CRISTIAN PATRICIO,Desconocido,GUAYAQUIL,GUAYAS
4,ABALCO HARO CRISTIAN PATRICIO,ASISTENCIA DE CONTABILIDAD,06-02-2022,8,79,ABALCO HARO CRISTIAN PATRICIO,Desconocido,GUAYAQUIL,GUAYAS
...,...,...,...,...,...,...,...,...,...
48145,WINNING CONSULTORES S A,GESTIÓN ESPECIALIZADA EN VENTAS,28-02-2020,5,307,WINNING CONSULTORES S A,FINALIZO VIGENCIA DE RECONOCIMIENTO,Sin Cantón,Sin Provincia
48146,WINNING CONSULTORES S A,GESTIÓN ESPECIALIZADA EN VENTAS,29-02-2020,6,307,WINNING CONSULTORES S A,FINALIZO VIGENCIA DE RECONOCIMIENTO,Sin Cantón,Sin Provincia
48147,WINNING CONSULTORES S A,MAQUILLAJE,17-02-2020,1,307,WINNING CONSULTORES S A,FINALIZO VIGENCIA DE RECONOCIMIENTO,Sin Cantón,Sin Provincia
48148,WINNING CONSULTORES S A,ORGANIZACIÓN Y GESTIÓN DE LA SECRETARIA,03-02-2020,1,307,WINNING CONSULTORES S A,FINALIZO VIGENCIA DE RECONOCIMIENTO,Sin Cantón,Sin Provincia


In [ ]:

rhs = (df_demanda_oec.Perfil
          .apply(lambda x: df_perfil[df_perfil.Perfil_Perfil.str.find(x).ge(0)]['id_perfil'])
          .bfill(axis=1)
          .iloc[:, 0])

#df_demanda_oec = pd.merge(df_demanda_oec,df_perfil,left_on=['OEC','Perfil'],right_on=['Perfil_Razon_Social_OEC','Perfil_Perfil'],how="left")



In [ ]:
df_res = pd.concat([df_demanda_oec, rhs], axis=1, ignore_index=True)
res_columns = df_demanda_oec.columns
res_columns = np.append(res_columns.values, 'id_perfil')
df_res = pd.DataFrame(data=df_res.values,columns=res_columns)


In [ ]:
df_res.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 48150 entries, 0 to 48149
Data columns (total 10 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   OEC                  48150 non-null  object
 1   Perfil               48150 non-null  object
 2   Fecha_Certificacion  48150 non-null  object
 3   num_cer              48150 non-null  object
 4   id_oec               47252 non-null  object
 5   razon_social_oec     47252 non-null  object
 6   estado               47252 non-null  object
 7   canton               47252 non-null  object
 8   provincia            47252 non-null  object
 9   id_perfil            21590 non-null  object
dtypes: object(10)
memory usage: 3.7+ MB


In [ ]:
df_demanda_oec = df_res[['id_oec','id_perfil','Fecha_Certificacion','num_cer']]

In [ ]:
df_fecha = df_demanda_oec['Fecha_Certificacion'].drop_duplicates()
df_fecha = df_fecha.rename_axis('id_fecha').reset_index().astype('object')

In [ ]:
df_demanda_oec = pd.merge(df_demanda_oec,df_fecha,left_on=['Fecha_Certificacion'],right_on=['Fecha_Certificacion'],how="left")



In [ ]:
df_demanda_oec[['id_oec','id_perfil','id_fecha','num_cer']]
df_demanda_oec.to_csv(path_datawarehouse+path_datamart_oec+'/demanda_oec.csv',index=False)

In [ ]:
data_fecha = df_fecha['Fecha_Certificacion'].str.rsplit("-", expand=True)
df_fecha = pd.concat([df_fecha,data_fecha],axis=1)

In [ ]:

df_fecha = df_fecha.drop('Fecha_Certificacion',axis=1)


In [ ]:
df_fecha = df_fecha.rename(columns={0:'dia',1:'mes',2:'año'})
df_fecha.to_csv(path_datawarehouse+path_datamart_oec+'/fecha.csv',index=False)

In [ ]:
df_demanda_oec = df_pcoec.groupby(['OEC','Perfil','Fecha_Certificacion']).count().reset_index().iloc[:,:4]
df_demanda_oec = df_demanda_oec.rename(columns={'Nombres':'num_cer'})
df_demanda_oec = pd.merge(df_demanda_oec,df_razon_social_oec,left_on=['OEC'],right_on=['nombre_oec'],how="left")
df_demanda_oec = pd.merge(df_demanda_oec,df_perfil,left_on=['OEC','Perfil'],right_on=['Razon_Social','Perfil'],how="left")
df_demanda_oec = df_demanda_oec[['id_oec','id_perfil','Fecha_Certificacion','num_cer']]
df_demanda_oec.info()


KeyError: ignored

# Dataframe CI

In [45]:
os.mkdir(path_datawarehouse+path_datamart_ci,0o666)

In [13]:
df_capacitador_ci = pd.read_csv(path+'CI_Provincia_Canton.csv')
df_capacitador_ci = df_capacitador_ci.rename(columns={'Nombre':'razon_social_ci'})
df_capacitador_ci  = df_capacitador_ci.rename_axis('id_ci').reset_index().astype('object')
df_capacitador_ci

,id_ci,razon_social_ci,Provincia,Cantón
0,0,ABATA CARRERA GLORIA BEATRIZ,ESMERALDAS,ESMERALDAS
1,1,ABATA CARRERA GLORIA BEATRIZ,ESMERALDAS,ESMERALDAS
2,2,ABRIL VELEZ ERIKA LUCIA,PICHINCHA,PUERTO QUITO
3,3,ACOSTA CAMINO DIEGO FERNANDO,TUNGURAHUA,AMBATO
4,4,ACOSTA CORDOVA ANDRES EDUARDO,PICHINCHA,QUITO
...,...,...,...,...
1614,1614,ZHIÑA VILLA DARIO XAVIER,AZUAY,CUENCA
1615,1615,ZORRILLA CERTUCHE MARIO ANDRES,PICHINCHA,QUITO
1616,1616,ZUMARRAGA PROAÑO BERTHA OLIVA,PICHINCHA,QUITO
1617,1617,ZURIAGA ANCHUNDIA MARIA JOSE,LOS RIOS,QUEVEDO


In [36]:
df_ci_lca = pd.read_csv(path+'ci_lca.csv')
df_ci_lca =  df_ci_lca.rename_axis('id_curso').reset_index().astype('object')
df_ci_lca

,id_curso,Numero_Documento,Apellidos_Nombres,Nombre_Curso,Area,Especialidad,Carga Horaria
0,0,914804364,VALENZUELA MENDIETA RICHARD OSWALDO,PREVENCIÓN DE RIESGOS LABORALES,PROCESOS INDUSTRIALES,SEGURIDAD PREVENCIÓN DE RIESGOS E HIGIENE INDU...,16.0
1,1,914804364,VALENZUELA MENDIETA RICHARD OSWALDO,INDICADORES DEL SISTEMA DE GESTIÓN DE LA SEGUR...,PROCESOS INDUSTRIALES,SEGURIDAD PREVENCIÓN DE RIESGOS E HIGIENE INDU...,40.0
2,2,1710803394,PEREZ SISA FERNANDO GUSTAVO,ANÁLISIS FINANCIERO,FINANZAS COMERCIO Y VENTAS,RIESGO FINANCIERO,40.0
3,3,1710803394,PEREZ SISA FERNANDO GUSTAVO,GESTIÓN FINANCIERA Y PRESUPUESTARIA EMPRESARIAL,FINANZAS COMERCIO Y VENTAS,RIESGO FINANCIERO,40.0
4,4,104364112,PEÑA ENCALADA PABLO ANDRES,CUIDADOS TERAPÉUTICOS DEL ADULTO MAYOR,SERVICIOS SOCIOCULTURALES Y A LA COMUNIDAD,SALUD Y MEDICINA,40.0
...,...,...,...,...,...,...,...
2929,2929,803114982,VELASCO CAICEDO CARLOS LUIS,REVIT ARQUITECTURA,CONSTRUCCIÓN E INFRAESTRUCTURA,TECNOLOGÍA DE LA CONSTRUCCIÓN,40.0
2930,2930,1803947769,ALDAS SALAZAR DARWIN SANTIAGO,USO DE NORMAS APA EN INVESTIGACIONES CIENTÍFIC...,TECNOLOGÍAS DE LA INFORMACIÓN Y COMUNICACIÓN,PROGRAMAS DE ESCRITORIO,24.0
2931,2931,922329057,MARQUEZ DIAZ KARINA MONICA,DERECHO ADMINISTRATIVO,ADMINISTRACIÓN Y LEGISLACIÓN,LEGISLACIÓN,40.0
2932,2932,922329057,MARQUEZ DIAZ KARINA MONICA,DERECHOS HUMANOS CON ÉNFASIS EN PROTECCIÓN INT...,ADMINISTRACIÓN Y LEGISLACIÓN,LEGISLACIÓN,32.0


In [42]:
df_pcci = pd.read_csv(path+'pcci.csv')
df_pcci = df_pcci.groupby(['Instructor_Capacitador','Nombre_Curso']).count().reset_index()[['Instructor_Capacitador','Nombre_Curso','Numero_Documento']]
df_pcci = df_pcci.rename(columns={'Numero_Documento':'num_cap'})
df_pcci = pd.merge(df_pcci,df_capacitador_ci,left_on=['Instructor_Capacitador'],right_on=['razon_social_ci'],how="left")
df_pcci = pd.merge(df_pcci,df_ci_lca,left_on=['Instructor_Capacitador','Nombre_Curso'],right_on=['Apellidos_Nombres','Nombre_Curso'],how="left")
demanda_ci = df_pcci[['id_ci','id_curso','num_cap']]

In [43]:
demanda_ci.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2686 entries, 0 to 2685
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   id_ci     1788 non-null   object
 1   id_curso  1937 non-null   object
 2   num_cap   2686 non-null   int64 
dtypes: int64(1), object(2)
memory usage: 83.9+ KB


In [47]:
df_capacitador_ci.to_csv(path_datawarehouse+path_datamart_ci+'/razon_social_ci.csv',index=False)
df_ci_lca.to_csv(path_datawarehouse+path_datamart_ci+'/curso_ci.csv',index=False)
demanda_ci.to_csv(path_datawarehouse+path_datamart_ci+'/demanda_ci.csv',index=False)